# High-level Keras (CNTK) Example

In [1]:
# Parameters
EPOCHS=5
N_CLASSES=10
BATCHSIZE=64
LR=0.01
MOMENTUM=0.9
GPU=True

In [2]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "cntk"
import keras as K
import cntk
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from utils import cifar_for_library, yield_mb
from nb_logging import NotebookLogger, output_to, error_to
from os import path

Using CNTK backend


In [3]:
nb_teminal_logger = NotebookLogger(sys.stdout.session, sys.stdout.pub_thread, sys.stdout.name, sys.__stdout__)

In [4]:
rst_out = output_to(nb_teminal_logger)
rst_err = error_to(nb_teminal_logger)

In [5]:
# channels_first is faster
K.backend.set_image_data_format('channels_first')

In [6]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print(K.backend.backend())
# Check that channels_last is selected (otherwise slow)
print(K.backend.image_data_format())

OS:  linux
Python:  3.5.4 |Anaconda, Inc.| (default, Oct  5 2017, 08:00:22) 
[GCC 7.2.0]
Keras:  2.0.8
Numpy:  1.13.3
CNTK:  2.1
cntk
channels_first


In [7]:
data_path = path.join(os.getenv('AZ_LEARNING_INPUT_DATASET'), 'cifar-10-batches-py')

In [8]:
def create_symbol():
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(3, 32, 32)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(N_CLASSES, activation='softmax'))
    return model

In [9]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [10]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(data_path, channel_first=True, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...


Preparing test set...
Done.


(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 844 ms, sys: 584 ms, total: 1.43 s
Wall time: 1.42 s


In [11]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 940 ms, sys: 696 ms, total: 1.64 s
Wall time: 1.76 s


In [12]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.19 ms


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 32, 32)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 32, 32)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 16, 16)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 16, 16)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 100, 8, 8)         0         
__________

In [14]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/5


   64/50000 [..............................] - ETA: 895s - loss: 2.3103 - acc: 0.0312

  128/50000 [..............................] - ETA: 609s - loss: 2.3018 - acc: 0.0625

  256/50000 [..............................] - ETA: 313s - loss: 2.3023 - acc: 0.0859

  384/50000 [..............................] - ETA: 229s - loss: 2.3024 - acc: 0.0859

...

49472/50000 [============================>.] - ETA: 0s - loss: 0.8450 - acc: 0.7015

49664/50000 [============================>.] - ETA: 0s - loss: 0.8452 - acc: 0.7014

49856/50000 [============================>.] - ETA: 0s - loss: 0.8448 - acc: 0.7015

50000/50000 [==============================] - 18s - loss: 0.8448 - acc: 0.7016    


CPU times: user 42.8 s, sys: 28.9 s, total: 1min 11s
Wall time: 1min 47s


In [15]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 512 ms, sys: 352 ms, total: 864 ms
Wall time: 1.56 s


In [16]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.7274
